In [1]:
!pip install PyMuPDF pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 77.4 MB/s eta 0:00:00


In [2]:
import fitz  # PyMuPDF, our "fizzy" tool
import json
import os

def is_question_start(line: str) -> bool:
    """
    Checks if a line is the start of a new question without using regex.
    Identifies patterns like "1.", "2. ", " 15. ", etc.
    """
    clean_line = line.strip()
    dot_index = clean_line.find('.')

    # If there's no dot, or it's the first character, it's not a question start.
    if dot_index <= 0:
        return False

    # Get the part before the dot
    number_part = clean_line[:dot_index]

    # Check if that part consists only of digits
    return number_part.isdigit()

def is_option_label(line: str) -> bool:
    """
    Checks if a line is an option label like (A), (B), etc., without regex.
    """
    clean_line = line.strip()
    return (
        clean_line.startswith('(') and
        len(clean_line) >= 3 and
        clean_line[2] == ')' and
        'A' <= clean_line[1] <= 'Z'
    )

def extract_content(pdf_path: str, output_dir: str):
    """
    Extracts text and images from a PDF, organizes them by question,
    and saves the output in a structured JSON file.
    ** This version does NOT use the 're' (regex) library. **
    """
    # Create output directories if they don't exist
    images_dir = os.path.join(output_dir, "images")
    if not os.path.exists(images_dir):
        os.makedirs(images_dir)

    doc = fitz.open(pdf_path)
    all_questions = []
    current_question = None

    for page_num, page in enumerate(doc):
        # Extract images and their bounding boxes
        image_list = page.get_images(full=True)
        page_images = []
        for img_index, img in enumerate(image_list):
            xref = img[0]
            base_image = doc.extract_image(xref)
            image_bytes = base_image["image"]
            image_ext = base_image["ext"]

            img_rects = page.get_image_rects(xref)
            if not img_rects:
                continue

            image_rect = img_rects[0]
            image_path = os.path.join(images_dir, f"page{page_num+1}_image{img_index+1}.{image_ext}")

            with open(image_path, "wb") as f:
                f.write(image_bytes)

            page_images.append({"path": image_path, "rect": image_rect})

        # Extract text blocks
        blocks = page.get_text("dict", flags=11)["blocks"]

        # Group elements (text and images) by question
        for block in blocks:
            if "lines" in block:
                for line in block["lines"]:
                    line_text = "".join([span["text"] for span in line["spans"]]).strip()
                    if not line_text:
                        continue

                    # --- MODIFICATION START: Replaced regex with function call ---
                    if is_question_start(line_text):
                    # --- MODIFICATION END ---
                        if current_question:
                            all_questions.append(current_question)

                        current_question = {
                            "question": line_text,
                            "images": "",
                            "option_images": []
                        }
                    elif current_question:
                        # --- MODIFICATION START: Replaced regex with function call ---
                        if not is_option_label(line_text):
                        # --- MODIFICATION END ---
                            current_question["question"] += " " + line_text

        # Associate images with the most recently defined question
        if current_question and page_images:
            text_content = page.get_text()
            has_options = any(opt in text_content for opt in ['(A)', '(B)', '(C)', '(D)'])

            if not has_options or len(page_images) <= 1:
                question_images = page_images
                option_images = []
            else:
                question_images = [page_images.pop(0)]
                option_images = page_images

            if question_images:
                current_question["images"] = question_images[0]["path"]

            if option_images:
                current_question["option_images"] = [img["path"] for img in option_images]

    if current_question:
        all_questions.append(current_question)

    json_path = os.path.join(output_dir, "extracted_content.json")
    with open(json_path, "w") as f:
        json.dump(all_questions, f, indent=4)

    print(f"Extraction complete.")
    print(f"Images saved in: {images_dir}")
    print(f"JSON data saved to: {json_path}")


if __name__ == "__main__":
    PDF_FILE = "/content/IMO class 1 Maths Olympiad Sample Paper 1 for the year 2024-25.pdf"
    OUTPUT_DIRECTORY = "output_no_regex"

    if not os.path.exists(PDF_FILE):
        print(f"Error: The file '{PDF_FILE}' was not found.")
        print("Please make sure the PDF file is in the same directory as the script.")
    else:
        extract_content(PDF_FILE, OUTPUT_DIRECTORY)

Extraction complete.
Images saved in: output_no_regex/images
JSON data saved to: output_no_regex/extracted_content.json
